# ETL Process

Para la realización de limpieza y acomodación del csv, utilizaremos la librería pandas de python.

In [3]:
import pandas as pd

## Extracción

Se lee el csv y se muestran los 5 primeros registros para visualizar el formato de dataframe.

In [30]:
df = pd.read_csv("movies.csv")
df.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


## Transformación

De los campos a transformar se seleccionaron 'Director', 'Genre' y 'Actors'. Siendo los ultimos 2 listas de 3 elementos separados por coma. Para ello se utilizará la función propia de pandas
        
        pd.get_dummies(sep='*')

La cual recibe como parametro el caracter por el cual separar los elementos y retorna un dataframe con los valores separados con columnas y su representación binaria o vectorial.

### Ejemplo

Se generará el dataframe para la columna 'Year' y se mostrarán solamente los primeros 5 elementos

In [34]:
pd.get_dummies(data=df['Year']).head()

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,1


Es así como generaremos 3 dataframe diferentes, uno para cada atributo a transformar ('Director', 'Genre' y 'Actors')

In [ ]:
# Generar dummies para genero, director y actores
df_genre = df["Genre"].str.get_dummies(sep=',')
df_director = df["Director"].str.get_dummies(sep=',')
df_actors = df["Actors"].str.get_dummies(sep=',')

Posterior a la generación de los dataframes, se concatenarán los dataframes necesarios para dar formato al vector que ingresaremos al KD Tree. Esta será:
* **Primera posición**: id o 'Rank' 
* **Segundo orden**: generos
* **Tercer orden**: director
* **Cuarto orden**: actores
* **Quinto orden**: rating

In [35]:
# Guardamos en dataframe para generar un vector
df_vector = pd.concat( [df["Rank"], df_genre, df_director, df_actors, df["Rating"] ], axis=1 )
df_vector.head()

,Rank,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,...,Woody Allen,Xavier Samuel,Yoo Gong,Zac Efron,Zach Galifianakis,Zachary Gordon,Zachary Quinto,Zoey Deutch,Zooey Deschanel,Rating
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8.1
1,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.3
3,4,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,7.2
4,5,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.2


## Cargar

Para trabajar el vector de posición en el árbol, se trabajará como lista, para ello solamente exportamos los valores del dataframe a una lista con el metodo incluido en pandas como se muestra a continuación.

In [41]:
# v[0] : "Rank" | v[1:] : point
lista_vectores = df_vector.values.tolist()

## Otras funcionalidades

Para buscar la información de las películas, se ocupará el dataframe, ya que el nodo del KD Tree sólo almacena el valor del id. Para buscar los datos se buscará como a continuación:

In [37]:
df_vector[ df_vector["Rank"] == 234]

,Rank,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,...,Woody Allen,Xavier Samuel,Yoo Gong,Zac Efron,Zach Galifianakis,Zachary Gordon,Zachary Quinto,Zoey Deutch,Zooey Deschanel,Rating
233,234,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.4
